# 8.4 질병이 있는 잎사귀 식별 경진대회 성능 개선
- [질병이 있는 잎사귀 식별 경진대회 링크](https://www.kaggle.com/c/plant-pathology-2020-fgvc7)
- [모델링 코드 참고 링크](https://www.kaggle.com/akasharidas/plant-pathology-2020-in-pytorch)

In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드 값 고정
seed = 10

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(train, 
                                test_size=0.1,
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']],
                                random_state=10)

In [5]:
import cv2
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스
import numpy as np

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__() # 상속받은 Dataset의 __init__() 메서드 호출
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    # 데이터 세트 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0] # 이미지 ID
        img_path = self.img_dir + img_id + '.jpg' # 이미지 파일 경로
        image = cv2.imread(img_path) # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        # 이미지 변환
        if self.transform is not None:
            image = self.transform(image=image)['image']
        # 테스트 데이터면 이미지 데이터만 반환, 그렇지 않으면 타깃 값(label)도 함께 반환
        if self.is_test:
            return image
        else:
            # 타깃 값 4개 중 가장 큰 값의 인덱스
            label = np.argmax(self.df.iloc[idx, 1:5]) 
            return image, label

In [6]:
# 이미지 변환을 위한 모듈
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(400, 600), # 이미지 크기 조절
    # 밝기 또는 대비 조절 
    A.OneOf([A.RandomBrightness(limit=0.1, p=1), 
             A.RandomContrast(limit=0.1, p=1)]),
    A.VerticalFlip(p=0.5), # 상하 대칭 변환
    A.HorizontalFlip(p=0.5), # 좌우 대칭 변환
    # 이동, 스케일링, 회전 변환 
    A.ShiftScaleRotate(
        shift_limit=0.2,
        scale_limit=0.2,
        rotate_limit=20, p=1),
    # 양각화, 날카로움, 블러 효과 
    A.OneOf([A.IAAEmboss(p=1),
             A.IAASharpen(p=1),
             A.Blur(p=1)], p=0.5),
    A.IAAPiecewiseAffine(p=0.5), # 어파인 변환 
    A.Normalize(), # 정규화 변환 
    ToTensorV2() # 텐서로 변환
])

# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(400, 600),
    A.Normalize(),
    ToTensorV2()
])

In [7]:
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [8]:
from torch.utils.data import DataLoader # 데이터 로더 생성을 위한 클래스

batch_size = 4

loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

## 8.4.1 모델 생성 및 훈련, 성능 검증

In [9]:
!pip install efficientnet-pytorch==0.7.1

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=9b8b1414de9d1f26dfb55ab918ba942a10d07cfee3b2a0d24a103d6c117271dc
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [10]:
from efficientnet_pytorch import EfficientNet # EfficientNet 모델

# 사전 훈련된 efficientnet-b7 모델 불러오기
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4) 

model = model.to(device) # 장비 할당

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


In [11]:
import torch.nn as nn # 신경망 모듈

# 손실함수
criterion = nn.CrossEntropyLoss()

In [12]:
# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0006, weight_decay=0.001)

### 스케줄러 설정

In [13]:
from transformers import get_cosine_schedule_with_warmup

epochs = 30 # 총 에폭

# 스케줄러
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*5, 
                                            num_training_steps=len(loader_train)*epochs)

### 모델 훈련 및 성능 검증

In [14]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수
from tqdm.notebook import tqdm # 진행률 표시 막대

# 총 에폭만큼 훈련
for epoch in range(epochs):
    model.train() # 모델을 훈련 상태로 설정
    epoch_train_loss = 0 # 에폭별 손실값 초기화 (훈련 데이터용)
    # 미니배치 크기만큼 데이터를 추출하는 작업을 '반복 횟수'만큼 반복 
    for images, labels in tqdm(loader_train):
        # 이미지, 레이블(타깃 값) 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        loss.backward() # 역전파 수행
        optimizer.step() # 가중치 갱신
        scheduler.step() # 스케줄러 학습률 갱신
        epoch_train_loss += loss.item() # 현재 배치에서의 손실 추가 (훈련 데이터용)
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
    
    model.eval() # 모델을 평가 상태로 설정 
    epoch_valid_loss = 0 # 에폭별 손실값 초기화 (검증 데이터용)
    preds_list = [] # 예측 확률값 저장용 리스트 초기화
    true_onehot_list = [] # 실제 타깃 값 저장용 리스트 초기화
    
    with torch.no_grad(): # 기울기 계산 비활성
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            
            preds = torch.softmax(outputs.cpu(), dim=1).numpy() # 예측 확률값
            true_onehot = torch.eye(4)[labels].cpu().numpy() # 실제값 (원-핫 인코딩 형식)
            # 예측 확률값과 실제값 저장
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
        # 검증 데이터 손실값 및 ROC AUC 점수 출력 
        print(f'에폭 [{epoch+1}/{epochs}] - 검증 데이터 손실값 : {epoch_valid_loss/len(loader_valid):.4f} / 검증 데이터 ROC AUC : {roc_auc_score(true_onehot_list, preds_list):.4f}')  

  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [1/30] - 훈련 데이터 손실값 : 0.9225
에폭 [1/30] - 검증 데이터 손실값 : 0.2904 / 검증 데이터 ROC AUC : 0.9245


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [2/30] - 훈련 데이터 손실값 : 0.4602
에폭 [2/30] - 검증 데이터 손실값 : 0.3768 / 검증 데이터 ROC AUC : 0.9622


  0%|          | 0/410 [00:00<?, ?it/s]

...

에폭 [28/30] - 훈련 데이터 손실값 : 0.0400
에폭 [28/30] - 검증 데이터 손실값 : 0.1818 / 검증 데이터 ROC AUC : 0.9882


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [29/30] - 훈련 데이터 손실값 : 0.0331
에폭 [29/30] - 검증 데이터 손실값 : 0.2235 / 검증 데이터 ROC AUC : 0.9877


  0%|          | 0/410 [00:00<?, ?it/s]

에폭 [30/30] - 훈련 데이터 손실값 : 0.0358
에폭 [30/30] - 검증 데이터 손실값 : 0.2049 / 검증 데이터 ROC AUC : 0.9874


## 8.4.2 예측 및 결과 제출

### TTA(테스트 단계 데이터 증강)

In [15]:
# 테스트 데이터 원본 데이터 세트 및 데이터 로더
dataset_test = ImageDataset(test, img_dir=img_dir, 
                            transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

# TTA용 데이터 세트 및 데이터 로더
dataset_TTA = ImageDataset(test, img_dir=img_dir, 
                           transform=transform_train, is_test=True)
loader_TTA = DataLoader(dataset_TTA, batch_size=batch_size, shuffle=False)

### 예측

In [16]:
model.eval() # 모델을 평가 상태로 설정 

preds_test = np.zeros((len(test), 4)) # 예측 값 저장용 배열 초기화

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        # 타깃 예측 확률
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
        preds_test[i*batch_size:(i+1)*batch_size] += preds_part

In [17]:
submission_test = submission.copy() # 제출 샘플 파일 복사

submission_test[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_test

In [18]:
num_TTA = 5 # TTA 횟수

preds_tta = np.zeros((len(test), 4)) # 예측 값 저장용 배열 초기화 (TTA용)

# TTA를 적용해 예측
for i in range(num_TTA):
    with torch.no_grad():
        for i, images in enumerate(loader_TTA):
            images = images.to(device)
            outputs = model(images)
            # 타깃 예측 확률
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

In [19]:
preds_tta /= num_TTA 

In [20]:
submission_tta = submission.copy() 

submission_tta[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_tta

### 결과 제출

In [21]:
submission_test.to_csv('submission_test.csv', index=False)
submission_tta.to_csv('submission_tta.csv', index=False)

### 레이블 스무딩

In [22]:
def apply_label_smoothing(df, target, alpha, threshold):
    # 타깃 값 복사
    df_target = df[['healthy', 'multiple_diseases', 'rust', 'scab']].copy()
    k = len(target) # 타깃 값 개수
    
    for idx, row in df_target.iterrows():
        if (row > threshold).any(): # 임계값을 넘는 타깃 값인지 여부 판단
            row = (1 - alpha)*row + alpha/k # 레이블 스무딩 적용  
            df_target.iloc[idx] = row # 레이블 스무딩을 적용한 값으로 변환
    return df_target # 레이블 스무딩을 적용한 타깃 값 반환

In [23]:
alpha = 0.01 # 레이블 스무딩 강도
threshold = 0.99 # 레이블 스무딩을 적용할 임계값

# 레이블 스무딩을 적용하기 위해 DataFrame 복사
submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()
target = ['healthy', 'multiple_diseases', 'rust', 'scab'] # 타깃 값 열 이름

# 레이블 스무딩 적용
submission_test_ls[target] = apply_label_smoothing(submission_test_ls, target, 
                                                   alpha, threshold)
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target, 
                                                  alpha, threshold)

submission_test_ls.to_csv('submission_test_ls.csv', index=False)
submission_tta_ls.to_csv('submission_tta_ls.csv', index=False)

In [24]:
alpha = 0.035 # 레이블 스무딩 강도
threshold = 0.98 # 레이블 스무딩을 적용할 임계값

# 레이블 스무딩을 적용하기 위해 DataFrame 복사 
submission_test_ls = submission_test.copy()
submission_tta_ls = submission_tta.copy()

# 레이블 스무딩 적용
submission_test_ls[target] = apply_label_smoothing(submission_test_ls, target, 
                                                   alpha, threshold)
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target, 
                                                  alpha, threshold)

submission_test_ls.to_csv('submission_test_ls2.csv', index=False)
submission_tta_ls.to_csv('submission_tta_ls2.csv', index=False)

In [25]:
path = './' # 모델 저장 경로

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
    }, path + 'EfficientNet-B7.tar')